In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install accelerate -U
!pip install evaluate
!pip install datasets
!pip install pymorphy2

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import nltk
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
import re
from datasets import Dataset, DatasetDict
import datasets
from transformers import Trainer, TrainingArguments, set_seed
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
from transformers import DataCollatorWithPadding
import evaluate
from transformers import EvalPrediction
import os
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
import pymorphy2

##Подготовка данных

In [ ]:
# считываем размеченные данные
df = pd.read_csv("")

print(df.shape)
df.head(3)

(409887, 2)


,text,predicted_class
0,получение куар хода,Проблема с получением
1,потеряли код получения заказа,Код получения
2,получить посылку,Проблема с получением


In [ ]:
# переводим классы в формат чисел
def classifactions(data):
  if data == 'Проблема с получением':
    return 1
  elif data == 'Код получения':
    return 2
  elif data == 'Соедините с оператором':
    return 3
  elif data == 'Адрес доставки':
    return 4
  elif data == 'Посылка не пришла':
    return 5
  elif data == 'Статус заказа':
    return 6
  elif data == 'Получатель подтверждает что заберт заказ':
    return 7
  elif data == 'Получить информацию о заказе':
    return 8
  elif data == 'Проблема с открытием ячейки':
    return 9
  elif data == 'Не работает постамат':
    return 10
  elif data == 'Получатель не сможет забрать заказ':
    return 11
  elif data == 'Узнать дату получения':
    return 12
  elif data == 'Местонахождение заказа':
    return 13
  elif data == 'Продлить срок хранения':
    return 14
  elif data == 'Узнать срок хранения':
    return 15
  elif data == 'Сотрудник магазина':
    return 16
  elif data == 'Срок доставки':
    return 17
  elif data == 'Номер заказа':
    return 18
  elif data == 'Unknown Cluster':
    return 19
  else:
    return 20

df['class'] = df.predicted_class.apply(lambda x: classifactions(x))

In [ ]:
# делим данные на обучающую и тестовую выборки
# тест будем использовать для проверки модели на отложенной выборке при необходимости
train, test = train_test_split(df, test_size=0.3)

In [ ]:
# удаляем лишние колонки
train = train.drop('predicted_class', axis=1)
test = test.drop('predicted_class', axis=1)

,text,class
14082,код посылки не приходит,2
208850,вход по адресу гражданская сто семнадцать,4
187012,куар код посылки,2
134482,приехал получать заказ а заказа нет,11
238192,мы отказались от заказа,1
...,...,...
156169,хочу узнать статус заказа номер эр пи пятьдеся...,6
334600,а товар пришел а в пункт выдачи в постамат по ...,5
193892,так заказ шестьдесят восемь четырнадцать я вам...,1
386647,а мне нужен код выдачи по моему заказу,4


In [ ]:
# очистка текстовых данных
def preprocess_text(data, stopwords=stopwords.words('russian')):
  """ Очистка текста и удаление стоп слов """
  text = re.sub('ё', 'е', data[0])
  text = ''.join([char for char in text if char not in string.punctuation])  # Удаление знаков препинания
  text = ' '.join(text.split())  # Удаление повторяющихся пробелов
  text = text.strip()
  # удаление стоп слов
  text = [w for w in text.split() if w.lower() not in stopwords]
  # удаление слов короче 3 символов
  text  = [w for w in text if len(w) >=3]
  return ' '.join(text)

# нормализация текстовых данных
morph = pymorphy2.MorphAnalyzer()
def lemmatization_text(data, morph=morph):
  """ Приведение слов к нормальной форме """
  words = data.split()
  normalized_words = [morph.parse(word)[0].normal_form for word in words]
  return ' '.join(normalized_words)


KeyboardInterrupt: 

In [ ]:
# чистим обучающую и тестовую выборки
train['text_clean'] = train['text'].apply(preprocess_text)
train['text_clean'] = train['text'].apply(lemmatization_text)

test['text_clean'] = test['text'].apply(preprocess_text)
test['text_clean'] = test['text'].apply(lemmatization_text)


In [ ]:
# удаляем не нужные столбцы
train = train.drop('text', axis=1)
test = test.drop('text', axis=1)
# преименовываем колонки
 в удобное название
train.rename(columns={'text_clean':'text'}, inplace=True)
test.rename(columns={'text_clean':'text'}, inplace=True)

train.rename(columns={'class':'label'}, inplace=True)
test.rename(columns={'class':'label'}, inplace=True)

In [ ]:
# делим на тренировчный и валидационный датасет
df_train, df_valid= train_test_split(train[['text', 'label']], test_size=0.2)

In [ ]:
# создаем словарь для подачи в модель
dataset = DatasetDict({
    "train": Dataset.from_pandas(df_train.reset_index(drop = True)),
    "valid": Dataset.from_pandas(df_valid.reset_index(drop = True)),
    'test': Dataset.from_pandas(test_upd)
    })
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 229536
    })
    valid: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 57384
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 122967
    })
})

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
# токенизируем текстовые данные
def preprocess_data(examples):

    encoding = tokenizer(examples['text'])
    return encoding

# используем эмбэндинги DeepPavlov/rubert-base-cased
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding = True)

encoded_dataset = dataset.map(preprocess_data, batched = True)
encoded_dataset = encoded_dataset.remove_columns(['text'])
encoded_dataset = encoded_dataset.rename_column("label", "labels")
encoded_dataset.set_format("torch")
encoded_dataset

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/229536 [00:00<?, ? examples/s]

Map:   0%|          | 0/57384 [00:00<?, ? examples/s]

Map:   0%|          | 0/122967 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 229536
    })
    valid: Dataset({
        features: ['Unnamed: 0', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 57384
    })
    test: Dataset({
        features: ['Unnamed: 0', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 122967
    })
})

## Обучение модели

In [ ]:
# определяем метрики качества
ACCURACY = evaluate.load("accuracy")
MCC = evaluate.load("matthews_correlation")

# функция для расчета метрик
def compute_metrics(p: EvalPrediction):
    preds = p.predictions
    preds = np.argmax(preds, axis=1)

    acc_result = ACCURACY.compute(predictions=preds, references=p.label_ids)
    mcc_result = MCC.compute(predictions=preds, references=p.label_ids)
    result = {"accuracy": acc_result["accuracy"], "mcc": mcc_result["matthews_correlation"]}

    return result


metric = evaluate.load("accuracy")

In [ ]:
# загружаем модель
model = AutoModelForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased", num_labels = 20)

training_args = TrainingArguments(
    output_dir=f"Roberta",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    weight_decay=0.1,
    num_train_epochs=3,
    warmup_ratio=0.1,
    optim="adamw_torch",
    save_strategy="no",
    save_total_limit=1,
    group_by_length=True,
    report_to="none"

)

# обучаем модель
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['valid'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Mcc
1,0.304700,0.342430,0.880855,0.854777
2,0.225400,0.229650,0.923045,0.902764
